In [1]:
import os


In [2]:
%pwd

'd:\\TextSummarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\TextSummarizer'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path 

In [24]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [26]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size 

In [31]:
import os
import urllib.request as request
from pathlib import Path
import zipfile

from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size  # assuming you wrote get_size() in utils

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")


In [32]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-08-18 16:33:09,104: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-18 16:33:09,106: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-18 16:33:09,107: INFO: common: created directory at: artifacts]
[2025-08-18 16:33:09,108: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-18 16:33:09,109: INFO: 4126061965: File already exists of size: ~ 7718 KB]
[2025-08-18 16:33:09,238: INFO: 4126061965: Extracted artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
